In [10]:
# I chose the Exponential Smoothing (Holt-Winters) model

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# load data
TRAIN_URL = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
TEST_URL  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

train_df = pd.read_csv(TRAIN_URL, parse_dates=["Timestamp"])
test_df  = pd.read_csv(TEST_URL,  parse_dates=["Timestamp"])


train_df.columns = train_df.columns.str.strip()
test_df.columns  = test_df.columns.str.strip()

# train
train_df = (
    train_df
    .sort_values("Timestamp")
    .set_index("Timestamp")
    .asfreq("H")
)

y_train = pd.to_numeric(train_df["trips"], errors="coerce").interpolate(limit_direction="both")

y_train = y_train.ffill().bfill()

#build and fit
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated"
)
modelFit = model.fit(optimized=True, use_brute=True)

#  pred
h = len(test_df)
forecast = modelFit.forecast(steps=h)
pred = np.asarray(forecast).ravel()

# print
print("len(pred) =", len(pred))
print("dtype(pred[0]) =", type(pred[0]))
print("NaNs in pred:", np.isnan(pred).sum())




/tmp/ipython-input-234823271.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .asfreq("H")


len(pred) = 744
dtype(pred[0]) = <class 'numpy.float64'>
NaNs in pred: 0
